In [1]:
import numpy as np
import keras
from keras.layers import *
from keras.models import Model, Sequential
from keras.optimizers import RMSprop
import load
import sys
import random
import os
from load import sentence2vector
from load import vector2sentence


Using TensorFlow backend.


In [2]:
def attention_3d_block(inputs, output_dim):
    # inputs.shape = (batch_size, time_steps, input_dim)

    a = TimeDistributed(Dense(1, activation='tanh'))(inputs)
    a = Flatten()(a)
    a = Activation('softmax')(a)
    a = RepeatVector(output_dim)(a)
    a = Permute([2, 1])(a)

    sent_representation = merge([inputs, a], mode='mul')
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)

    return sent_representation


In [3]:

def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    pass


In [4]:
def one_hot_encoder(value_vector, max_type = 5,one_index=False):
    one_hot_matrix = np.zeros([value_vector.shape[0],max_type],dtype=np.uint8)
    for i,score in enumerate(value_vector):
        one_hot_matrix[i,score-int(one_index)] = 1
    return one_hot_matrix


In [ ]:
iteration=60
preprocessor = load.Preprocessor(r'./Data')
preprocessor.load_char_embedding(filename='clean_embedding.txt')
print(len(preprocessor.char2indice))
print((preprocessor.char_embedding_matrix.shape))
X,Score, Y = load.build_X_Y_score('sample.json',preprocessor)

In [5]:
MAX_SEQUENCE_LENGTH = 32
LSTM_UNITS=256
preprocessor = load.Preprocessor(r'./Data')
# preprocessor.load_char_embedding(filename='clean_embedding.txt')
preprocessor.load_word_indice('word_count_10.tsv')
print(len(preprocessor.word2indice))
# print(len(preprocessor.char_embedding_matrix.shape))
# model_build
output_shape=len(preprocessor.word2indice)

19092


In [ ]:

my_embedding_layer = Embedding(preprocessor.n_word,output_dim=128,
                               input_length=MAX_SEQUENCE_LENGTH)

a=Input(shape=(MAX_SEQUENCE_LENGTH,))
x1 = Masking(mask_value=0, input_shape=(MAX_SEQUENCE_LENGTH,))(a)
x1 = my_embedding_layer(x1)
x1 = LSTM(units=LSTM_UNITS, activation='tanh',return_sequences=True)(x1)
print(x1.shape)
# x1 = LSTM(units=512, activation='tanh')(x1)
x1 = attention_3d_block(x1,LSTM_UNITS)
print(x1.shape)
x = Dense(512, activation='relu')(x1)
x = Dropout(0.2)(x)

b = Input(shape=(5,))
x = concatenate([x, b])
x = Dense(output_shape, activation='softmax')(x)

# x = Dropout(0.3)(x)
model = Model(inputs=[a, b], outputs=x)
# optimizer = RMSprop(0.003)

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
#train
# earlyStop=keras.callbacks.EarlyStopping(monitor='val_acc',mode='max',patience=20)
# saveBestModel=keras.callbacks.ModelCheckpoint(filepath='lstm_attention.{epoch:02d-{val_acc:.2f}}.hdf5',monitor='val_acc',save_best_only=True,mode='max')
iteration=20
for _ in range(iteration):
    
    
    fold=20
    data_generator = load.build_X_Y_score('review_word.tsv',preprocessor=preprocessor,fold=fold,duplicate=10 ,max_len=MAX_SEQUENCE_LENGTH,begin=-32,stop=-30)
    for i in range(fold):
        print('\n','iteration,fold:',_+1,i)
        X,Score,Y = next(data_generator)
        #print(X.shape,Score.shape,Y.shape)
        Y=load.one_hot_encoder(Y,len(preprocessor.word2indice),one_index=False)
        model.fit(x=[X,Score],y=Y,batch_size=64,epochs=1,validation_split=0.1,shuffle=True,)
        
    model.save('./models/lstm_attention_iteration%d_units_%d_val-acc_%.3f.h5'%(_+1,LSTM_UNITS,model.history.history['val_acc'][0]))
# model.save('LSTM_baseline.h5')

In [6]:
model=keras.models.load_model('./models/lstm_attention_iteration8_units_256_val-acc_0.314.h5')

C:\Users\asus\Anaconda3\lib\site-packages\keras\engine\topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [27]:
score=5
print()
print('----- score:', score)

# sentence_indice=list(np.ones([MAX_SEQUENCE_LENGTH],dtype=np.uint32)*0)
# sentence_indice[-1]=2
seed='很 好吃 ， 尤其是 招牌 鸡 … 吃饭 之间 还 有'
vector=seed.split(' ')
sentence_vector =  ['padding']*(MAX_SEQUENCE_LENGTH-len(vector))+vector
sentence_vector = sentence2vector(sentence_vector,preprocessor)
# generated = ''.join(vector2sentence(sentence_vector,preprocessor))
#2代表的是<sor>的indice
score_one_hot=np.zeros([1,5])
score_one_hot[0,score-1]=1
b=score_one_hot
generated_vector=[]
while (True):
    
    a=np.array([sentence_vector])
    print("a.shape",a.shape)
    softmax_vector=model.predict([a,b])[0]
    
    next_index = sample(softmax_vector, temperature=0.7)
    print(next_index)
    if next_index == 3:
        break
    sentence_vector.pop(0)
    sentence_vector.append(next_index)
    generated_vector.append(next_index)
    
# print(sentence_vector)
generated=''.join(vector2sentence(generated_vector,preprocessor))
print(''.join(seed.split(' ')),generated)


----- score: 5
a.shape (1, 32)
7102
a.shape (1, 32)
12744
a.shape (1, 32)
9694
a.shape (1, 32)
7067
a.shape (1, 32)
3
很好吃，尤其是招牌鸡…吃饭之间还有 学生用餐的。
